Here do also the t-test for the results of the different models by metric.

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
from tabulate import tabulate

In [2]:
def display_structure(dictionary):
    rows = []
    for key, value in dictionary.items():
        if isinstance(value, dict):
            for subkey, subvalue in value.items():
                if isinstance(subvalue, dict):
                    for subsubkey, subsubvalue in subvalue.items():
                        rows.append([key, subkey, subsubkey, type(subsubvalue).__name__])
                        key = ''
                        subkey = ''
                else:
                    rows.append([key, subkey, '', type(subvalue).__name__])
                    key = ''
                    subkey = ''
        else:
            rows.append([key, '', '', type(value).__name__])
    print(tabulate(rows, headers=['Key', 'Subkey', 'Sub-subkey', 'Type']))

In [5]:
def display_results(dictionary, detailed=False):
    rows = []
    config_rows = []
    f1_rows = []
    r2_rows = []
    pearson_rows = []
    average_rows = []
    for key, value in dictionary.items():
        if key == 'config':
            target_rows = config_rows
        elif key == 'f1':
            target_rows = f1_rows
            
            # aggregate secondary f1 scores
            f1_secondary_means = [v['mean'] for k,v in value.items() if k != 'target']
            f1_secondary_stds = [v['std'] for k,v in value.items() if k != 'target']
            
        elif key == 'r2':
            target_rows = r2_rows
        elif key == 'pearson':
            target_rows = pearson_rows
        elif key == 'average':
            target_rows = average_rows
        else:
            target_rows = rows
        if isinstance(value, dict):
            for subkey, subvalue in value.items():
                if isinstance(subvalue, dict):
                    for subsubkey, subsubvalue in subvalue.items():
                        target_rows.append([key, subkey, subsubkey, format_value(subsubvalue)])
                        key = ''
                        subkey = ''
                else:
                    target_rows.append([key, subkey, '', format_value(subvalue)])
                    key = ''
        else:
            target_rows.append([key, '', '', format_value(value)])
    
    # add f1 secondary scores
    average_rows.append(['', 'secondary f1', 'mean', format_value(np.mean(f1_secondary_means))])
    average_rows.append(['', '', 'std', format_value(np.mean(f1_secondary_stds))])

    if not detailed:
        final_rows = config_rows + average_rows
    else:
        final_rows = config_rows + average_rows + f1_rows + r2_rows + pearson_rows + rows
    print(
        tabulate(
            final_rows, # + rows, 
            headers=['Key', 'Subkey', 'Sub-subkey', 'Value']
        )
    )
    #return average_rows

def format_value(value):
    if isinstance(value,(int,float)):
        return f'{value:.2f}'
    elif isinstance(value,list):
        return f'{len(value)} cls'
    else:
        return value

In [57]:
# import any json file from the results folders

file = "./results/targCls_2_music_openl3_env_voice_True_TotCls_5_dropNs_True_rep_5_fold_5.json"

with open(file) as f:
    data = json.load(f)
    print(file)
    display_results(data, detailed=True)
    #print(data["config"]["classifications"].keys())


./results/targCls_2_music_openl3_env_voice_True_TotCls_5_dropNs_True_rep_5_fold_5.json
Key      Subkey                    Sub-subkey         Value
-------  ------------------------  -----------------  ----------
config   modality                                     music
         voice                                        1.00
         classifications           voice_gender       4 cls
                                   voice_exagg        4 cls
                                   voice_type         4 cls
                                   voice_age          4 cls
                                   target             2 cls
         which_embeddings                             openl3_env
         drop_non_significant                         1.00
average  mid_r2                    mean               0.35
                                   std                0.12
         mid_pears                 mean               0.62
                                   std                0.08
         

Target f1 
    - with
    - without voice
Avg. secondary f1
    - with
    - without voice
Avg. R2 emotions
    - with
    - without voice
Avg. r emotions
    - with
    - without voice
Avg. R2 mid-level
    - with
    - without voice
Avg. r mid-level
    - with
    - without voice

In [73]:
columns = ["Embeddings" ,"Target F1", "Avg. secondary F1", "Avg. R2 emotions", "Avg. r emotions", "Avg. R2 mid-level", "Avg. r mid-level"]

lvl_1 = ["Binary", "Ternary"]

lvl_2 = ["Voice", "No voice"]

dict_of_dfs = {k1: {k2: pd.DataFrame(columns=columns) for k2 in lvl_2} for k1 in lvl_1}

for fn in glob.glob("./results/*.json"):
    
    with open(fn) as f:
        results = json.load(f)

    # aggregate secondary f1 scores
    f1_secondary_means = [v['mean'] for k,v in results["f1"].items() if k != 'target']
    f1_secondary_stds = [v['std'] for k,v in results["f1"].items() if k != 'target']
    results["average"]["secondary f1"] = { "mean": np.mean(f1_secondary_means), "std": np.mean(f1_secondary_stds)}

    first_level = "Binary" if len(results["config"]["classifications"]["target"]) == 2 else "Ternary"
    second_level = "Voice" if results["config"]["voice"] else "No voice"

    row = {
        "Embeddings": results["config"]["which_embeddings"],
        "Target F1": f'{results["f1"]["target"]["mean"]:.2f} ± {results["f1"]["target"]["std"]:.2f}',
        "Avg. secondary F1": f'{results["average"]["secondary f1"]["mean"]:.2f} ± {results["average"]["secondary f1"]["std"]:.2f}',
        "Avg. R2 emotions": f'{results["average"]["emo_r2"]["mean"]:.2f} ± {results["average"]["emo_r2"]["std"]:.2f}',
        "Avg. r emotions": f'{results["average"]["emo_pears"]["mean"]:.2f} ± {results["average"]["emo_pears"]["std"]:.2f}',
        "Avg. R2 mid-level": f'{results["average"]["mid_r2"]["mean"]:.2f} ± {results["average"]["mid_r2"]["std"]:.2f}',
        "Avg. r mid-level": f'{results["average"]["mid_pears"]["mean"]:.2f} ± {results["average"]["mid_pears"]["std"]:.2f}',
        

    }

    df = dict_of_dfs[first_level][second_level].copy()
    df = df.append(row, ignore_index=True)
    dict_of_dfs[first_level][second_level] = df




In [79]:
# aggregate secondary f1 scores
f1_secondary_means = [v['mean'] for k,v in results["f1"].items() if k != 'target']
f1_secondary_stds = [v['std'] for k,v in results["f1"].items() if k != 'target']
results["average"]["secondary f1"] = { "mean": np.mean(f1_secondary_means), "std": np.mean(f1_secondary_stds)}

first_level = "Binary" if len(results["config"]["classifications"]["target"]) == 2 else "Ternary"
second_level = "Voice" if results["config"]["voice"] else "No voice"

row = {
    "Embeddings": results["config"]["which_embeddings"],
    "Target F1": f'{results["f1"]["target"]["mean"]:.2f} ± {results["f1"]["target"]["std"]:.2f}',
    "Avg. secondary F1": f'{results["average"]["secondary f1"]["mean"]:.2f} ± {results["average"]["secondary f1"]["std"]:.2f}',
    "Avg. R2 emotions": f'{results["average"]["emo_r2"]["mean"]:.2f} ± {results["average"]["emo_r2"]["std"]:.2f}',
    "Avg. r emotions": f'{results["average"]["emo_pears"]["mean"]:.2f} ± {results["average"]["emo_pears"]["std"]:.2f}',
    "Avg. R2 mid-level": f'{results["average"]["mid_r2"]["mean"]:.2f} ± {results["average"]["mid_r2"]["std"]:.2f}',
    "Avg. r mid-level": f'{results["average"]["mid_pears"]["mean"]:.2f} ± {results["average"]["mid_pears"]["std"]:.2f}',
    

}

df = dict_of_dfs[first_level][second_level].copy()
df = df.append(row, ignore_index=True)



/tmp/ipykernel_162590/3711744804.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)


In [82]:
df

,Target F1,Avg. secondary F1,Avg. R2 emotions,Avg. r emotions,Avg. R2 mid-level,Avg. r mid-level
Embeddings,,,,,,
mfcc,0.79 ± 0.08,0.66 ± 0.07,0.02 ± 0.17,0.36 ± 0.11,0.14 ± 0.16,0.48 ± 0.09


In [60]:
dict_of_dfs["Binary"]["Vanilla"]["Voice"]

,Target F1,Avg. secondary F1,Avg. R2 emotions,Avg. r emotions,Avg. R2 mid-level,Avg. r mid-level


In [9]:
# Create a DataFrame with nested columns
iterables = [['A', 'B'], ['x', 'y']]
columns = pd.MultiIndex.from_product(iterables, names=['first', 'second'])
df = pd.DataFrame([[1, 2, 3, 4], [5, 6, 7, 8]], columns=columns)

# Convert the DataFrame to a LaTeX table
latex_table = df.to_latex(index=False)

# Write the LaTeX table to a file
with open('table.tex', 'w') as f:
    f.write(latex_table)

/tmp/ipykernel_162590/2702114544.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=False)


In [23]:
def df_to_latex(df, file_name):
    # Determine the number of columns in the DataFrame
    num_columns = len(df.columns)

    # Create a column format string with centered columns
    column_format = 'c' * num_columns

    # Convert the DataFrame to a LaTeX table with centered headers
    latex_table = df.to_latex(index=False, column_format=column_format, multicolumn=True, multicolumn_format='c')

    # Write the LaTeX table to a file
    with open(file_name, 'w') as f:
        f.write(latex_table)

In [29]:
# Create a DataFrame with multi-level columns and unique labels in the top level
columns = pd.MultiIndex.from_arrays([['A', '', 'B', ''], ['x', 'y', 'x', 'y']])
df = pd.DataFrame([[1, 2, 3, 4], [5, 6, 7, 8]], columns=columns)

# Convert the DataFrame to a LaTeX table
latex_table = df.to_latex(index=False, multicolumn=True, multicolumn_format='c')

# Write the LaTeX table to a file
with open('table.tex', 'w') as f:
    f.write(latex_table)

# Display the resulting LaTeX table
print(latex_table)

\begin{tabular}{rrrr}
\toprule
A & \multicolumn{2}{c}{B} \\
x & y & x & y \\
\midrule
1 & 2 & 3 & 4 \\
5 & 6 & 7 & 8 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_162590/3352871230.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=False, multicolumn=True, multicolumn_format='c')


In [48]:
df = pd.DataFrame(columns=['A x', "A y", "B x", "B y", "C x", "C y "], data=[[1,2,3,4,5,6],[5,6,7,8,9,10]])
# Convert the DataFrame to a LaTeX table
latex_table = df.to_latex(index=False, column_format='c'*len(df.columns))

/tmp/ipykernel_162590/2727934183.py:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=False, column_format='c'*len(df.columns))


In [49]:
df

,A x,A y,B x,B y,C x,C y
0,1,2,3,4,5,6
1,5,6,7,8,9,10


In [50]:
# Write the LaTeX table to a file
with open('table.tex', 'w') as f:
    f.write(latex_table)

# Display the resulting LaTeX table
print(latex_table)

\begin{tabular}{cccccc}
\toprule
 A x &  A y &  B x &  B y &  C x &  C y  \\
\midrule
   1 &    2 &    3 &    4 &    5 &     6 \\
   5 &    6 &    7 &    8 &    9 &    10 \\
\bottomrule
\end{tabular}



In [28]:
# Create a list of tuples representing the labels for each level of the MultiIndex
first_level = ['A', 'B', 'C']
second_level = ['x', 'y']

tuples = [(a, b) for a in first_level for b in second_level]

# Create a MultiIndex from the list of tuples
multiindex = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])

# Create a DataFrame with the MultiIndex as columns
df = pd.DataFrame([[1, 2, 3, 4, 5, 6], [7,8,9,10,11,12]], columns=multiindex)

# Display the resulting DataFrame
print(df)

# Write the DataFrame to a LaTeX table
df_to_latex(df, 'table.tex')

first   A     B       C    
second  x  y  x   y   x   y
0       1  2  3   4   5   6
1       7  8  9  10  11  12


/tmp/ipykernel_162590/57950410.py:9: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=False, column_format=column_format, multicolumn=True, multicolumn_format='c')
